# Pearson heat maps

Creates pearson heat maps for host gal properties and pew or velocity.

Creates a dataframe of pearson values which are significant.

## imports

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import MultipleLocator
from scipy.stats.stats import pearsonr
from pandas import DataFrame

In [5]:
    feature_alias = {
        'pW1': 'Ca ii H&K',
        'pW2': 'Si ii λ4130',
        'pW3': 'Mg ii, Fe ii',
        'pW4': 'Fe ii, Si ii',
        'pW5': 'S ii λ5449, λ5622',
        'pW6': 'Si ii λ5972',
        'pW7': 'Si ii λ6355',
        'pW8': 'Ca ii IR triplet'}

In [1]:
sdss_combined = pd.read_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/residual_df.csv', index_col=['feat_name', 'obj_id'])
sdss_combined = sdss_combined.iloc[: , 1:]
sdss_combined

NameError: name 'pd' is not defined

In [ ]:
def pearson_list(df, features, xcol, ycol):
    """Creates lists of pearson statistic p and c values for each feature for any two given columns
        used in pearson_table_make function
    
    Args:
        df     (DataFrame): The dataframe in which the columns we want to compare are
        features (List[str]): The feature ids to add to table
        xcol           (str): Name of one column to compare
        ycol           (str): Name of second column to compare
    
    Returns:
        p_list   (List[float]): List of P values
        c_list   (List[float]): List of c values
    """
    
    p_list = []
    c_list = []
    
    for feat in zip(features):
        feat_data = df.loc[feat]
        x = feat_data[xcol]
        y = feat_data[ycol]
        
        no_nan = (np.isfinite(x) & np.isfinite(y))
        
        if len(x[no_nan]) >= 3:
            corr, p = pearsonr(x[no_nan], y[no_nan])
            p_list.append(p)
            c_list.append(corr)
            
        else:
            p_list.append(np.nan)
            c_list.append(np.nan)
        
    return p_list, c_list
        

In [ ]:
def pearson_table_make(df, features):
    """Creates data frame of p and c pearson stat values for specific columns vs. pew and vel
    
    Args:
        df     (DataFrame): The dataframe in which the columns we want to compare are
        features (List[str]): The feature ids to add to table 
        
    Returns:
        data    (DataFrame): Table of p and c values for different qualities vs pew and vel.
    """
        
    data = pd.DataFrame()
    data['features'] = features
    
    df = df_col = df[['zCMB','zspecHelio','logMassFSPS','logSSFRFSPS',
                      'ageFSPS','kpc', 'residuals', 'pew','vel']]
    df_col = df[['zCMB','zspecHelio','logMassFSPS','logSSFRFSPS'
                 ,'ageFSPS','kpc', 'residuals']]
    
    for column in df_col:
        p_pew, c_pew = pearson_list(df, features, column , 'pew')
        data[f'{column}_c_pew'] = c_pew
        data[f'{column}_p_pew'] = p_pew
        p_vel, c_vel = pearson_list(df, features, column, 'vel')
        data[f'{column}_c_vel'] = c_vel
        data[f'{column}_p_vel'] = p_vel
    
    data = data.set_index('features')
    return data

In [ ]:
features = [f'pW{i}' for i in range(1, 9)]
pearson_table = pearson_table_make(sdss_combined, features)
pearson_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/pearson_table.csv')

pd.set_option("display.max_rows", None, "display.max_columns", None)

#test
pearson_table.iloc[:,8:]
itable = pearson_table.iloc[:,8:]
ptable = itable[itable.columns[1::2]]
ptable_pew = ptable[ptable.columns[::2]]
ptable_pew

## creating entries for heatmap

In [ ]:
def heat_map_entry(table):
    """Create arrays to enter into the plot_heat_map function
    Args:
        table     (DataFrame): The data to use
    Returns:
        carray_pew    (array): array of c values against pew
        parray_pew    (array): array of p values against pew
        carray_vel    (array): array of c values against vel
        parray_vel    (array): array of p values against vel
    """
 
    # keeps only features 1-7
    table = table.iloc[:7,:]
    # creates indexed table
    itable = table.set_index('features')
    # keeps properties we want to plot
    itable = itable.iloc[:,8:]

    # creates dfs of just c and p values and just pew and vel
    ctable = itable[itable.columns[::2]]
    ptable = itable[itable.columns[1::2]]
    ctable_pew = ctable[ctable.columns[::2]]
    ctable_vel = ctable[ctable.columns[1::2]]
    ptable_pew = ptable[ptable.columns[::2]]
    ptable_vel = ptable[ptable.columns[1::2]]
    
    #makes dfs into arrays
    carray_pew = ctable_pew.to_numpy()
    carray_vel = ctable_vel.to_numpy()
    parray_pew = ptable_pew.to_numpy()
    parray_vel = ptable_vel.to_numpy()

    return carray_pew, parray_pew, carray_vel, parray_vel

In [ ]:
features = [f'pW{i}' for i in range(1, 8)]
properties = ['logMassFSPS', 'logSSFRFSPS',
              'ageFSPS','kpc', 'residuals']

# heatmap plotting functions

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    #ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
def plot_heat_map(carray, parray, against):
    """plots 2 heat maps for c and p value respectively
       host gal properties on vertical axis and features on horizontal
    Args:
        carray     (array): array of c values
        parray     (array): array of p values
        against    (string): pew or vel
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    # 1st plot


    im, _ = heatmap(carray, features, properties, ax=ax1,
                cmap="coolwarm", cbarlabel=f"pearson correlation values for property vs. {against}")
    annotate_heatmap(im, valfmt="{x:.3f}", size=10, color=('black'))

    # 2nd plot

    im, _ = heatmap(parray, features, properties, ax=ax2, vmin=0,
                cmap="Blues_r", cbarlabel=f"p values for property vs. {against}")
    annotate_heatmap(im, valfmt="{x:.3f}", size=10, threshold=0.6,
                 textcolors=("white","black"))

    plt.tight_layout()
    plt.show()

In [ ]:
sdss_cl = sdss_combined[sdss_combined['branch_type'] == 'CL']
sdss_bl = sdss_combined[sdss_combined['branch_type'] == 'BL']
sdss_cn = sdss_combined[sdss_combined['branch_type'] == 'CN']
sdss_ss = sdss_combined[sdss_combined['branch_type'] == 'SS']

In [ ]:
features = [f'pW{i}' for i in range(1, 9)]
bl_pearson_table = pearson_table_make(sdss_bl, features)
bl_pearson_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/bl_pearson_table.csv')

In [ ]:
features = [f'pW{i}' for i in range(1, 9)]
cn_pearson_table = pearson_table_make(sdss_cn, features)
cn_pearson_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/cn_pearson_table.csv')

In [ ]:
features = [f'pW{i}' for i in range(1, 8)]
cl_pearson_table = pearson_table_make(sdss_cl, features)
cl_pearson_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/cl_pearson_table.csv')

In [ ]:
features = [f'pW{i}' for i in range(1, 8)]
ss_pearson_table = pearson_table_make(sdss_ss, features)
ss_pearson_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/ss_pearson_table.csv')

# heat maps for all SNe

In [ ]:
#table = pd.read_csv("/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/pearson_table.csv")
#table = pd.read_csv("/Users/ellakane/Desktop/research/SN-Spectral-Evolution/results/pearson_table.csv")
table = pearson_table.reset_index()
carray_pew, parray_pew, carray_vel, parray_vel = heat_map_entry(table)

In [ ]:
plot_heat_map(carray_pew, parray_pew, 'pew')

In [ ]:
plot_heat_map(carray_vel, parray_vel, 'vel')

# for broad line

In [ ]:
bl_table = pd.read_csv("/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/bl_pearson_table.csv")
bl_carray_pew, bl_parray_pew, bl_carray_vel, bl_parray_vel = heat_map_entry(bl_table)

In [ ]:
plot_heat_map(bl_carray_pew, bl_parray_pew, 'pew BL')

In [ ]:
plot_heat_map(bl_carray_vel, bl_parray_vel, 'vel BL')

# for shallow silicon

In [ ]:
ss_table = pd.read_csv("/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/ss_pearson_table.csv")
ss_carray_pew, ss_parray_pew, ss_carray_vel, ss_parray_vel = heat_map_entry(ss_table)

In [ ]:
plot_heat_map(ss_carray_pew, ss_parray_pew, 'pew SS')

In [ ]:
plot_heat_map(ss_carray_vel, ss_parray_vel, 'vel SS')

# for cool

In [ ]:
cl_table = pd.read_csv("/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/cl_pearson_table.csv")
cl_carray_pew, cl_parray_pew, cl_carray_vel, cl_parray_vel = heat_map_entry(cl_table)

In [ ]:
plot_heat_map(cl_carray_pew, cl_parray_pew, 'pew CL')

In [ ]:
plot_heat_map(cl_carray_vel, cl_parray_vel, 'vel CL')

# for core normal

In [ ]:
cn_table = pd.read_csv("/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/cn_pearson_table.csv")
cn_carray_pew, cn_parray_pew, cn_carray_vel, cn_parray_vel = heat_map_entry(cn_table)

In [ ]:
plot_heat_map(cn_carray_pew, cn_parray_pew, 'pew CN')

In [ ]:
plot_heat_map(cn_carray_vel, cn_parray_vel, 'vel CN')

In [ ]:
def significant_list(table):
    """Create significant pearson values dataframe by both c and p cutoffs
    Args:
        table     (DataFrame): The data to use
    Returns:
        df        (DataFrame): dataframe of significant pearson values  
    """
 
    # keeps only features 1-7
    table = table.iloc[:7,:]
    # creates indexed table
    itable = table.set_index('features')
    #im not sure if we need to set index or not!!
    #itable = table
    # keeps properties we want to plot
    itable = itable.iloc[:,8:]
    # creates dfs of just c and p values and just pew and vel
    ctable = itable[itable.columns[::2]].reset_index()
    ictable = itable[itable.columns[::2]]
    cprops = ctable.columns.values.tolist()
    ptable = itable[itable.columns[1::2]].reset_index()
    iptable = itable[itable.columns[1::2]]
    pprops = ptable.columns.values.tolist()
    
    name_list = []
    feat_list = []
    c_list = []
    p_list = []
    
    #breakpoint()
    for i, j in zip(cprops[1:], pprops[1:]):
        ccolumn = ictable[i]
        pcolumn = iptable[j]
        cname = i
        pname = j
   
        for c, p in zip(ccolumn, pcolumn):
            cindex = ictable[ictable[cname]==c].index.values
            if ((np.abs(c) >= 0.4) and (p <= 0.05)):
                name_list.append(cname)
                feat_list.append(cindex)
                c_list.append(c)
                p_list.append(p)
                #print(cname, cindex, c, p)
            else:
                continue
    data = {'property_comparison':name_list, 'feature':feat_list, 'c':c_list, 'p':p_list}
    df = pd.DataFrame(data, columns=['property_comparison','feature','c','p'])
    return df

In [ ]:
def significant_p_list(table):
    """Create significant pearson values dataframe by just p cutoffs
    Args:
        table     (DataFrame): The data to use
    Returns:
        df        (DataFrame): dataframe of significant pearson values
    """
 
    # keeps only features 1-7
    table = table.iloc[:7,:]
    # creates indexed table
    itable = table.set_index('features')
    #im not sure if we need to set index or not!!
    #itable = table
    # keeps properties we want to plot
    itable = itable.iloc[:,8:]
    # creates dfs of just c and p values and just pew and vel
    ctable = itable[itable.columns[::2]].reset_index()
    ictable = itable[itable.columns[::2]]
    cprops = ctable.columns.values.tolist()
    ptable = itable[itable.columns[1::2]].reset_index()
    iptable = itable[itable.columns[1::2]]
    pprops = ptable.columns.values.tolist()
    
    name_list = []
    feat_list = []
    c_list = []
    p_list = []
    
    #breakpoint()
    for i, j in zip(cprops[1:], pprops[1:]):
        ccolumn = ictable[i]
        pcolumn = iptable[j]
        cname = i
        pname = j
   
        for c, p in zip(ccolumn, pcolumn):
            cindex = ictable[ictable[cname]==c].index.values
            if np.abs(p) <= 0.05:
                name_list.append(cname)
                feat_list.append(cindex)
                c_list.append(c)
                p_list.append(p)
                #print(cname, cindex, c, p)
            else:
                continue
    data = {'property_comparison':name_list, 'feature':feat_list, 'c':c_list, 'p':p_list}
    df = pd.DataFrame(data, columns=['property_comparison','feature','c','p'])
    return df

### All data

In [ ]:
#df of significant trends for all data
sig_all = significant_list(table)
all_list = ['ALL']*len(sig_all['c'])
sig_all['data_used'] = all_list

In [ ]:
#df of only p significant trends for all data
sig_p_all = significant_p_list(table)
all_p_list = ['ALL']*len(sig_p_all['c'])
sig_p_all['data_used'] = all_p_list

In [ ]:
#df of significant trends for cl data only
sig_cl = significant_list(cl_table)
cl_list = ['CL only']*len(sig_cl['c'])
sig_cl['data_used'] = cl_list

In [ ]:
#df of only p significant trends for cl data only
sig_p_cl = significant_p_list(cl_table)
cl_p_list = ['CL only']*len(sig_p_cl['c'])
sig_p_cl['data_used'] = cl_p_list

In [ ]:
#df of significant trends for bl data only
sig_bl = significant_list(bl_table)
bl_list = ['BL only']*len(sig_bl['c'])
sig_bl['data_used'] = bl_list

In [ ]:
#df of only p significant trends for bl data only
sig_p_bl = significant_p_list(bl_table)
bl_p_list = ['BL only']*len(sig_p_bl['c'])
sig_p_bl['data_used'] = bl_p_list

In [ ]:
#df of significant trends for cn data only
sig_cn = significant_list(cn_table)
cn_list = ['CN only']*len(sig_cn['c'])
sig_cn['data_used'] = cn_list

In [ ]:
#df of only p significant trends for cn data only
sig_p_cn = significant_p_list(cn_table)
cn_p_list = ['CN only']*len(sig_p_cn['c'])
sig_p_cn['data_used'] = cn_p_list

In [7]:
#df of significant trends for ss data only
sig_ss = significant_list(ss_table)
ss_list = ['SS only']*len(sig_ss['c'])
sig_ss['data_used'] = ss_list

NameError: name 'significant_list' is not defined

In [ ]:
#df of only p significant trends for ss data only
sig_p_ss = significant_p_list(ss_table)
ss_p_list = ['SS only']*len(sig_p_ss['c'])
sig_p_ss['data_used'] = ss_p_list

### df of all significant trends where c >= 0.4 and p <= 0.05

In [ ]:
#sort lists by data used first, property second, feature third

In [ ]:
sig_df = sig_all.append(sig_cl, ignore_index = True)
sig_df = sig_df.append(sig_bl, ignore_index = True)
sig_df = sig_df.append(sig_cn, ignore_index = True)
sig_df = sig_df.append(sig_ss, ignore_index = True)
sig_df = sig_df.rename(columns = ({'c':'pearson correlation', 'p':'p value'}))
sig_df = sig_df.sort_values(by=['data_used', 'property_comparison'], ascending = True)
sig_df

### df of all significant trends where p<=0.05

In [ ]:
sig_p_df = sig_p_all.append(sig_p_cl, ignore_index = True)
sig_p_df = sig_p_df.append(sig_p_bl, ignore_index = True)
sig_p_df = sig_p_df.append(sig_p_cn, ignore_index = True)
sig_p_df = sig_p_df.append(sig_p_ss, ignore_index = True)
sig_p_df = sig_p_df.rename(columns = ({'c':'pearson correlation', 'p':'p value'}))
sig_p_df = sig_p_df.sort_values(by=['data_used', 'property_comparison'], ascending = True)
sig_p_df

In [ ]:
sdss_combined = sdss_combined.reset_index()

In [ ]:
def pearson_list_prop(df, xcol):
    """Creates lists of pearson statistic p and c values for each feature for any two given columns
        used in pearson_table_make function
    
    Args:
        df     (DataFrame): The dataframe in which the columns we want to compare are
        xcol           (str): Name of one column to compare
    
    Returns:
        p_list   (List[float]): List of P values
        c_list   (List[float]): List of c values
    """
    
    p_list = []
    c_list = []
    
    df = df.drop_duplicates('obj_id')

    x = df[xcol]
    y = df['residuals']
        
    no_nan = (np.isfinite(x) & np.isfinite(y))
        
    corr, p = pearsonr(x[no_nan], y[no_nan])
    p_list.append(p)
    c_list.append(corr)
        
    return p_list, c_list
        

In [ ]:
def pearson_table_make_prop(df):
    """Creates data frame of p and c pearson stat values for specific columns vs. residuals
    
    Args:
        df     (DataFrame): The dataframe in which the columns we want to compare are
        
    Returns:
        data    (DataFrame): Table of p and c values for different qualities vs residuals
    """
        
    data = pd.DataFrame()
    
    #df = df_col = df[['zCMB','zspecHelio','logMassFSPS','logSSFRFSPS',
                      #'ageFSPS','kpc', 'residuals']]
    df_col = df[['zCMB','zspecHelio','logMassFSPS','logSSFRFSPS'
                 ,'ageFSPS','kpc']]
    
    for column in df_col:
        p, c = pearson_list_prop(df, column)
        data[f'{column}_c'] = c
        data[f'{column}_p'] = p
    
    return data

In [ ]:
pearson_prop_table = pearson_table_make_prop(sdss_combined)
pearson_prop_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/pearson_prop_table.csv')
pearson_prop_table

In [ ]:
sdss_bl = sdss_bl.reset_index()
bl_pearson_prop_table = pearson_table_make_prop(sdss_bl)
bl_pearson_prop_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/bl_pearson_prop_table.csv')
bl_pearson_prop_table

In [ ]:
sdss_cl = sdss_cl.reset_index()
cl_pearson_prop_table = pearson_table_make_prop(sdss_cl)
cl_pearson_prop_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/cl_pearson_prop_table.csv')
cl_pearson_prop_table

In [ ]:
sdss_ss = sdss_ss.reset_index()
ss_pearson_prop_table = pearson_table_make_prop(sdss_ss)
ss_pearson_prop_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/ss_pearson_prop_table.csv')
ss_pearson_prop_table

In [ ]:
sdss_cn = sdss_cn.reset_index()
cn_pearson_prop_table = pearson_table_make_prop(sdss_cn)
cn_pearson_prop_table.to_csv('/Users/emilysf098/Desktop/Research/SN-Spectral-Evolution/results/cn_pearson_prop_table.csv')
cn_pearson_prop_table

In [ ]:
def heat_map_entry_prop(table):
    """Create arrays to enter into the plot_heat_map function
    Args:
        table     (DataFrame): The data to use

    """


    # creates dfs of just c and p values
    ctable = table[table.columns[::2]]
    ptable = table[table.columns[1::2]]
    
    #makes dfs into arrays
    carray = ctable.to_numpy()
    parray = ptable.to_numpy()

    return carray, parray

In [ ]:
c_array, p_array = heat_map_entry_prop(pearson_prop_table)

In [ ]:
c_array_bl, p_array_bl = heat_map_entry_prop(bl_pearson_prop_table)
c_array_cl, p_array_cl = heat_map_entry_prop(cl_pearson_prop_table)
c_array_cn, p_array_cn = heat_map_entry_prop(cn_pearson_prop_table)
c_array_ss, p_array_ss = heat_map_entry_prop(ss_pearson_prop_table)

In [ ]:
properties = ['zCMB', 'zspecHelio', 'logMassFSPS', 'logSSFRFSPS',
              'ageFSPS','kpc']

In [ ]:
def plot_heat_map_prop(carray, parray, against):
    """plots 2 heat maps for c and p value respectively
       host gal properties on vertical axis and features on horizontal
    Args:
        carray     (array): array of c values
        parray     (array): array of p values
        against    (string): pew or vel
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    # 1st plot


    im, _ = heatmap(carray.transpose(), properties, 'residuals', ax=ax1,
                cmap="coolwarm", cbarlabel=f"pearson correlation values for property vs. {against}")
    annotate_heatmap(im, valfmt="{x:.3f}", size=10, color=('white'))

    # 2nd plot

    im, _ = heatmap(parray.transpose(), properties, 'residuals', ax=ax2, vmin=0,
                cmap="Blues_r", cbarlabel=f"p values for property vs. {against}")
    annotate_heatmap(im, valfmt="{x:.3f}", size=10, threshold=0.1,
                 textcolors=("white","black"))

    plt.tight_layout()
    plt.show()

# All

In [ ]:
plot_heat_map_prop(c_array, p_array, 'residual')

# Broad Line

In [ ]:
plot_heat_map_prop(c_array_bl, p_array_bl, 'residual BL')

# Cool

In [ ]:
plot_heat_map_prop(c_array_cl, p_array_cl, 'residual CL')

# Core Normal

In [ ]:
plot_heat_map_prop(c_array_cn, p_array_cn, 'residual CN')

# Shallow Silicon

In [ ]:
plot_heat_map_prop(c_array_ss, p_array_ss, 'residual SS')